In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil import parser
%matplotlib inline


from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import GridSearchCV
import pickle
from lightgbm import LGBMClassifier
import requests
from contextlib import closing
import csv
print('Library Loaded')

Library Loaded


In [24]:
# data = './heart.csv'
## We want to read this in one row at a time; create a generator. 
url = "https://raw.githubusercontent.com/cosmicudemy/ML_Casestudies/master/heart%20disease/heart.csv"
with closing(requests.get(url, stream=True)) as r:
    """context manager to control closing"""
    f = (line.decode('utf-8') for line in r.iter_lines())
    reader = pd.read_csv(f, delimiter=',', quotechar='"')

    
    

# df = pd.read_csv("https://github.com/cosmicudemy/ML_Casestudies/blob/master/heart%20disease/heart.csv")
# df.shape


In [27]:
## This worked:
import io
def iterable_to_stream(iterable, buffer_size=io.DEFAULT_BUFFER_SIZE):
    class IterStream(io.RawIOBase):
        def __init__(self):
            self.leftover = None
        def readable(self):
            return True
        def readinto(self, b):
            try:
                l = len(b)
                chunk = self.leftover or next(iterable)
                output, self.leftover = chunk[:l], chunk[l:] # chunk, up to limit
                b[:len(output)] = output 
                return len(output)
            except StopIteration:
                return 0
    return io.BufferedReader(IterStream(), buffer_size=buffer_size)
    

In [29]:
response = requests.get(url, stream=True)

In [38]:
next(response.iter_content())

b'g'

In [39]:
df = pd.read_csv(iterable_to_stream(response.iter_content()), sep=",")

In [40]:
df

,e,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [41]:
# or
# file = "heart.csv"
# df_empty = pd.DataFrame()
# with open(file) as fl:
#     chunk_iter = pd.read_csv(fl, chunksize=10000)
#     for chunk in chunk_iter:
#         chunk = chunk[chunk["col1"] < 100]
#         df_empty = pd.concat([df_empty,chunk])


In [42]:
df.isnull().sum()

In [182]:
final_cols = ['cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal',
             ]

In [183]:
X = df.drop("target", axis=1)
from sklearn.manifold import TSNE
import time
time_start = time.time()

df_tsne = TSNE(random_state=10).fit_transform(X)
print("done {:.3f} seconds".format(time.time()-time_start))

done 2.083 seconds


In [198]:
y = df.target
X = df.drop("target",axis=1)
X_train , X_test , y_train , y_test = train_test_split(X, y, test_size = 0.10, random_state = 11)
print('Training Set :',len(X_train))
print('Test Set :',len(X_test))
print('Training labels :',len(y_train))
print('Test Labels :',len(y_test))

Training Set : 272
Test Set : 31
Training labels : 272
Test Labels : 31


In [145]:
from sklearn.impute import SimpleImputer

In [199]:
fill = SimpleImputer(missing_values = 0, strategy = "mean")

In [200]:
X_train = fill.fit_transform(X_train[final_cols])
X_test = fill.fit_transform(X_test[final_cols])

In [177]:
# X_train = pd.DataFrame(fill.fit_transform(X_train.iloc[:,2:]))
# X_test =  pd.DataFrame(fill.fit_transform(X_test.iloc[:,2:]))

In [201]:
X_train.shape

(272, 11)

In [202]:
X_test.shape

(31, 11)

In [203]:
def FitModel(X_train,y_train,X_test,y_test,algo_name,algorithm,gridSearchParams,cv):
    np.random.seed(10)
   
    grid = GridSearchCV(
        estimator=algorithm,
        param_grid=gridSearchParams,
        cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)
    
    
    grid_result = grid.fit(X_train, y_train)
    best_params = grid_result.best_params_
    pred = grid_result.predict(X_test)
    cm = confusion_matrix(y_test, pred)
   # metrics =grid_result.gr
    print(pred)
    pickle.dump(grid_result,open(algo_name,'wb'))
   
    print('Best Params :',best_params)
    print('Classification Report :',classification_report(y_test,pred))
    print('Accuracy Score : ' + str(accuracy_score(y_test,pred)))
    print('Confusion Matrix : \n', cm)

### Logistic regression (to be pickled and used)

In [204]:
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

FitModel(X_train,y_train,X_test,y_test,'LogisticRegression',LogisticRegression(),hyperparameters,cv=5)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[0 1 1 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1]
Best Params : {'C': 2.7825594022071245, 'penalty': 'l2'}
Classification Report :               precision    recall  f1-score   support

           0       0.83      0.71      0.77        14
           1       0.79      0.88      0.83        17

    accuracy                           0.81        31
   macro avg       0.81      0.80      0.80        31
weighted avg       0.81      0.81      0.80        31

Accuracy Score : 0.8064516129032258
Confusion Matrix : 
 [[10  4]
 [ 2 15]]


/Users/christophermartin/DocumentsNoCloud/repos/cmcode/deployml/predict_heart/venvheart/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.76478114        nan 0.77218855        nan 0.75737374
        nan 0.76114478        nan 0.76484848        nan 0.76491582
        nan 0.76484848        nan 0.75380471        nan 0.77218855
        nan 0.77218855]
  warnings.warn(
/Users/christophermartin/DocumentsNoCloud/repos/cmcode/deployml/predict_heart/venvheart/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [ ]:
## It's pickled, without .pkl extension; use it in the streamlit app. 